| **Informações**    |            **Detalhes**         |
|--------------------|---------------------------------|
| Nome da Tabela     |          Gold_Pedidos           |
| Data da Ingestao   |            31/03/2025           |
| Ultima Atualizaçao |            30/07/2025           |
| Origem             |         Silver.pedidos          |
| Responsável        |           Lucas Sousa           |
| Motivo             |   Criação de Camadas Gold       |
| Observações        |               None              |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |31/07/2025 | Lucas Sousa  | Criação do notebook |
 


###1. Desempenho de Vendas por Período
🎯 Objetivo: Compreender padrões sazonais e tendências de crescimento ou queda.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Esta análise fornece uma visão clara do progresso das vendas ao longo do tempo. É fundamental para a gestão financeira, pois permite que a equipe acompanhe o desempenho em tempo real, avalie a trajetória em relação a metas orçamentárias e identifique rapidamente se o negócio está no caminho certo para o sucesso anual. O uso do acumulado ajuda a suavizar as flutuações de curto prazo e a focar na tendência geral.</p>



In [0]:
display(spark.sql("""
WITH vendas_mensais AS (
    SELECT
        YEAR(data_pedido) AS ano,
        MONTH(data_pedido) AS mes,
        SUM(quantidade * preco) AS receita_mensal
    FROM silver.pedidos
    GROUP BY ano, mes
)
SELECT
    ano,
    mes,
    receita_mensal,
    SUM(receita_mensal) OVER (PARTITION BY ano ORDER BY mes) AS receita_acumulada_anual
FROM vendas_mensais
ORDER BY ano, mes
"""))

ano,mes,receita_mensal,receita_acumulada_anual
2023,7,33987.27,33987.27
2023,8,63758.35,97745.62
2023,9,81294.98,179040.60
2023,10,45259.91,224300.51
2023,11,66792.56,291093.07
2023,12,57310.15,348403.22
2024,1,55308.10,55308.10
2024,2,58914.52,114222.62
2024,3,57272.58,171495.20
2024,4,59854.70,231349.90


###2. Produtos Mais Rentaveis
🎯 Objetivo: Identificar os itens mais rentáveis para estratégias de promoção e precificação.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Identificar os produtos de maior sucesso em termos de receita é crucial para otimizar a estratégia de negócios. Com essa informação, a empresa pode focar em campanhas de marketing eficazes, planejar o estoque de forma mais assertiva e direcionar o desenvolvimento de novos produtos. Compreender o que o cliente mais valoriza permite alocar recursos de forma mais inteligente para maximizar a lucratividade.</p>

In [0]:
%sql
SELECT
    produto,
    SUM(quantidade * preco) AS receita_total_produto,
    SUM(quantidade) AS quantidade_total_vendida
FROM silver.pedidos
GROUP BY produto
ORDER BY receita_total_produto DESC
LIMIT 10

produto,receita_total_produto,quantidade_total_vendida
Vinho ABC,168253.57,14191
Cerveja XYZ,165041.51,15036
Água Mineral,135812.69,12874
Refrigerante 123,129574.58,11741
Suco Natural,126052.65,12931


###3. Desempenho por Estabelecimento
🎯 Objetivo: Reconhecer os parceiros mais lucrativos e direcionar campanhas de relacionamento.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Esta análise é vital para o gerenciamento da sua rede de parceiros. Ela permite premiar os melhores estabelecimentos (com bonificações, por exemplo) e identificar os de pior desempenho para oferecer suporte, treinamento ou, se necessário, reavaliar a parceria. É uma base sólida para a gestão de relacionamento e expansão da rede.</p>

In [0]:
%sql
SELECT
    nome_estabelecimento,
    SUM(quantidade * preco) AS receita_total,
    COUNT(DISTINCT id_pedido) AS numero_total_pedidos,
    SUM(quantidade) AS quantidade_total_itens_vendidos
FROM silver.pedidos
GROUP BY nome_estabelecimento
ORDER BY receita_total DESC

nome_estabelecimento,receita_total,numero_total_pedidos,quantidade_total_itens_vendidos
Padoca do Zé,40116.67,11,3077
Oficina Mecânica,32641.69,5,2188
Padaria do Paulo,29919.66,11,2361
Bar e Grill,27440.37,6,1902
Bar da Esquina,27249.56,7,2065
Farmácia Popular,25732.20,9,2678
Barbearia do Toninho,24500.19,8,2656
Cantina Italiana,23442.84,8,1998
Academia Fit,22054.23,8,2227
Bar do João,20065.57,8,1954


###4. Ticket Médio por Estabelecimento
🎯 Objetivo: Avaliar a lucratividade média por cliente e entender o comportamento de compra.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">O ticket médio é um indicador-chave da qualidade das vendas. Aumentar este valor significa que cada cliente está gerando mais receita por transação, o que pode ser resultado de estratégias de cross-selling (venda de produtos complementares) ou up-selling (venda de produtos de maior valor). Monitorar o ticket médio por estabelecimento e por período ajuda a identificar onde essas estratégias estão funcionando e quais estabelecimentos têm maior poder de venda.</p>

In [0]:
display(spark.sql("""
WITH valor_por_pedido AS (
    SELECT
        id_pedido,
        nome_estabelecimento,
        SUM(quantidade * preco) AS valor_do_pedido
    FROM silver.pedidos
    GROUP BY id_pedido, nome_estabelecimento
)
SELECT
    nome_estabelecimento,
    AVG(valor_do_pedido) AS ticket_medio
FROM valor_por_pedido
GROUP BY nome_estabelecimento
ORDER BY ticket_medio DESC
"""))


nome_estabelecimento,ticket_medio
Oficina Mecânica,6528.338000
Bar e Grill,4573.395000
Padaria do Bairro,4386.785000
Pastelaria da Vila,4371.440000
Adega do Luiz,4171.243333
Frutaria Tropical,4024.200000
Restaurante Mexicano,3989.690000
Restaurante Francês,3962.610000
Bar da Esquina,3892.794286
Botequim do Chico,3817.773333


###5. Estabelecimentos com Maior Número de Pedidos
🎯 Objetivo: Avaliar a popularidade e o engajamento dos estabelecimentos.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Esta métrica mede a popularidade e a capacidade de um estabelecimento de atrair e processar um grande volume de transações. Diferente da receita, ela foca na frequência e no engajamento dos clientes, sendo um indicador valioso para entender a capilaridade da sua marca.</p>

In [0]:
%sql
SELECT
    nome_estabelecimento,
    COUNT(DISTINCT id_pedido) AS numero_total_pedidos
FROM silver.pedidos
GROUP BY nome_estabelecimento
ORDER BY numero_total_pedidos DESC
LIMIT 10

nome_estabelecimento,numero_total_pedidos
Padaria do Paulo,11
Padoca do Zé,11
Farmácia Popular,9
Bar do João,8
Padaria São João,8
Barbearia do Toninho,8
Academia Fit,8
Cantina Italiana,8
Lanchonete Top,7
Bar da Esquina,7


###6. Mix de Produtos Vendidos por Estabelecimento
🎯 Objetivo: Identificar oportunidades de cross-selling e verificar a diversidade do portfólio.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">O "insight" que você mencionou está correto: entender a variedade de produtos que um estabelecimento vende ajuda a identificar oportunidades de cross-selling e a verificar a saúde e a diversidade do portfólio. Um estabelecimento com um mix variado provavelmente tem clientes mais engajados e pode ser um modelo a ser replicado.</p>

In [0]:
%sql
SELECT
    nome_estabelecimento,
    COUNT(DISTINCT produto) AS numero_produtos_unicos
FROM silver.pedidos
GROUP BY nome_estabelecimento
ORDER BY numero_produtos_unicos DESC


nome_estabelecimento,numero_produtos_unicos
Pizzaria Bella,5
Padaria São João,5
Padaria do Paulo,5
Barbearia do Toninho,5
Padoca do Zé,5
Cantina Italiana,5
Mercadinho da Esquina,4
Farmácia Popular,4
Cervejaria Artesanal,4
Loja de Roupas Fashion,4


###7. Tempo Médio entre Pedidos (Frequência de Compra)
🎯 Objetivo: Prever o churn (perda de clientes) e identificar clientes fiéis.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Este é um indicador avançado e crucial para prever o churn (perda de clientes) e identificar clientes fiéis. Clientes que compram com alta frequência são valiosos. Ao detectar um aumento no tempo entre as compras, a empresa pode agir de forma proativa, enviando ofertas personalizadas para reter o cliente.</p>

In [0]:
display(spark.sql("""
WITH pedidos_com_datas_anteriores AS (
    SELECT
        nome_estabelecimento,
        data_pedido,
        LAG(data_pedido, 1) OVER (PARTITION BY nome_estabelecimento ORDER BY data_pedido) AS data_pedido_anterior
    FROM silver.pedidos
),
diferenca_dias AS (
    SELECT
        nome_estabelecimento,
        DATEDIFF(data_pedido, data_pedido_anterior) AS dias_entre_pedidos
    FROM pedidos_com_datas_anteriores
    WHERE data_pedido_anterior IS NOT NULL
)
SELECT
    nome_estabelecimento,
    AVG(dias_entre_pedidos) AS tempo_medio_entre_pedidos
FROM diferenca_dias
GROUP BY nome_estabelecimento
ORDER BY tempo_medio_entre_pedidos ASC
"""))



nome_estabelecimento,tempo_medio_entre_pedidos
Botequim do Chico,10.5
Papelaria Central,14.0
Café Gourmet,23.0
Padoca do Zé,28.4
Padaria do Paulo,31.2
Sorveteria Sorvetão,33.666666666666664
Academia Fit,35.57142857142857
Cervejaria Gourmet,36.833333333333336
Bar da Esquina,37.333333333333336
Cantina Italiana,37.714285714285715


###8. Produtos com Baixa Saída
🎯 Objetivo: Gerenciar inventário e avaliar a eficiência do portfólio.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Conhecer os produtos com baixa saída é o lado oposto da análise de "mais vendidos". Este insight é fundamental para gerenciar o inventário, decidir se um produto deve ser descontinuado ou se precisa de uma promoção agressiva para escoar o estoque. É uma análise de eficiência de portfólio.</p>

In [0]:
%sql
SELECT
    produto,
    SUM(quantidade) AS quantidade_total_vendida_recente
FROM silver.pedidos
WHERE data_pedido >= DATE_SUB(current_date(), 90) -- Exemplo: últimos 90 dias
GROUP BY produto
ORDER BY quantidade_total_vendida_recente ASC
LIMIT 10


produto,quantidade_total_vendida_recente


###9. Análise de Crescimento Mensal (Mês-a-Mês)
🎯 Objetivo: Monitorar o ritmo de crescimento e reagir rapidamente a tendências.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Diferente do acumulado, esta análise compara a receita de cada mês com a do mês anterior. É uma métrica essencial para monitorar o ritmo de crescimento e reagir rapidamente a tendências de alta ou baixa. Ajuda a responder a perguntas como "O crescimento de vendas desacelerou em relação ao mês passado?"</p>

In [0]:
display(spark.sql("""
WITH receita_mensal AS (
    SELECT
        DATE_TRUNC('month', data_pedido) AS mes_ano,
        SUM(quantidade * preco) AS receita_mensal
    FROM silver.pedidos
    GROUP BY mes_ano
)
SELECT
    mes_ano,
    receita_mensal,
    LAG(receita_mensal, 1) OVER (ORDER BY mes_ano) AS receita_mes_anterior,
    (receita_mensal - LAG(receita_mensal, 1) OVER (ORDER BY mes_ano)) / LAG(receita_mensal, 1) OVER (ORDER BY mes_ano) * 100 AS crescimento_mensal_percentual
FROM receita_mensal
ORDER BY mes_ano DESC
"""))


mes_ano,receita_mensal,receita_mes_anterior,crescimento_mensal_percentual
2024-07-01T00:00:00Z,21589.58,36528.39,-40.896400
2024-06-01T00:00:00Z,36528.39,86863.91,-57.947600
2024-05-01T00:00:00Z,86863.91,59854.70,45.124600
2024-04-01T00:00:00Z,59854.70,57272.58,4.508500
2024-03-01T00:00:00Z,57272.58,58914.52,-2.787000
2024-02-01T00:00:00Z,58914.52,55308.10,6.520600
2024-01-01T00:00:00Z,55308.10,57310.15,-3.493400
2023-12-01T00:00:00Z,57310.15,66792.56,-14.196800
2023-11-01T00:00:00Z,66792.56,45259.91,47.575500
2023-10-01T00:00:00Z,45259.91,81294.98,-44.326300


###10. Variação de Preço por Produto
🎯 Objetivo: Detectar inconsistências ou oportunidades de ajustes comerciais.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Este insight ajuda a monitorar a consistência dos preços e a identificar possíveis problemas de dados ou inconsistências na estratégia de precificação. Uma variação significativa pode indicar promoções, diferenças entre fornecedores, ou até mesmo erros de registro. Monitorar isso garante a integridade dos dados e a estabilidade da sua estratégia de preços.</p>

In [0]:
display(spark.sql("""
SELECT
    produto,
    MIN(preco) AS preco_minimo,
    MAX(preco) AS preco_maximo,
    AVG(preco) AS preco_medio,
    (MAX(preco) - MIN(preco)) AS variacao_de_preco
FROM silver.pedidos
GROUP BY produto
ORDER BY variacao_de_preco DESC
"""))

produto,preco_minimo,preco_maximo,preco_medio,variacao_de_preco
Cerveja XYZ,1.30,19.92,10.835357,18.62
Refrigerante 123,1.28,19.87,11.213902,18.59
Suco Natural,1.72,19.99,9.867333,18.27
Água Mineral,1.81,19.92,10.070000,18.11
Vinho ABC,1.26,18.87,11.178182,17.61


###11. Vendas Acumuladas no Ano por Estabelecimento
🎯 Objetivo: Acompanhar o desempenho acumulado de cada parceiro ao longo do ano.

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Essa análise é uma extensão do "Faturamento por Estabelecimento" para a dimensão temporal. Ela permite que a gestão acompanhe o desempenho acumulado de cada parceiro ao longo do ano. Isso é útil para definir e monitorar metas de vendas para cada estabelecimento, além de identificar quem está se destacando na corrida anual.</p>

In [0]:
display(spark.sql("""
WITH vendas_mensais_estabelecimento AS (
    SELECT
        nome_estabelecimento,
        YEAR(data_pedido) AS ano,
        MONTH(data_pedido) AS mes,
        SUM(quantidade * preco) AS receita_mensal
    FROM silver.pedidos
    GROUP BY nome_estabelecimento, ano, mes
)
SELECT
    nome_estabelecimento,
    ano,
    mes,
    receita_mensal,
    SUM(receita_mensal) OVER (PARTITION BY nome_estabelecimento, ano ORDER BY mes) AS receita_acumulada_estabelecimento
FROM vendas_mensais_estabelecimento
ORDER BY nome_estabelecimento, ano, mes
"""))


nome_estabelecimento,ano,mes,receita_mensal,receita_acumulada_estabelecimento
Academia Fit,2023,10,1435.60,1435.60
Academia Fit,2023,11,441.25,1876.85
Academia Fit,2024,3,7196.22,7196.22
Academia Fit,2024,4,6342.34,13538.56
Academia Fit,2024,5,4314.01,17852.57
Academia Fit,2024,6,2324.81,20177.38
Adega do Luiz,2023,8,758.28,758.28
Adega do Luiz,2024,1,5983.25,5983.25
Adega do Luiz,2024,5,5772.20,11755.45
Bar da Esquina,2023,10,164.22,164.22


###12. Análise de Crescimento de Vendas Mês a Mês e Ano a Ano
🎯 Objetivo: Avaliar o desempenho de vendas no curto (mês a mês) e longo prazo (ano a ano).

<h4 style="color: #007bff; font-size: 1.2em;">Importância Estratégica:</h4>
<p style="font-size: 1.1em;">Esta é uma análise de crescimento abrangente. Ela permite avaliar o desempenho de vendas tanto no curto prazo (mês a mês) quanto no longo prazo (ano a ano), oferecendo uma visão completa da saúde do negócio. A capacidade de comparar o desempenho de um mês com o mesmo mês do ano anterior é crucial para eliminar a sazonalidade e entender o crescimento real da empresa.

In [0]:
display(spark.sql("""
WITH vendas_mensais AS (
    SELECT
        DATE_TRUNC('month', data_pedido) AS mes_ano,
        SUM(quantidade * preco) AS receita,
        SUM(quantidade) AS quantidade_vendas
    FROM silver.pedidos
    GROUP BY mes_ano
),
comparacao_anual AS (
    SELECT
        mes_ano,
        receita,
        quantidade_vendas,
        LAG(receita, 12, 0) OVER (ORDER BY mes_ano) AS receita_ano_anterior,
        LAG(quantidade_vendas, 12, 0) OVER (ORDER BY mes_ano) AS quantidade_ano_anterior
    FROM vendas_mensais
)
SELECT
    mes_ano,
    receita,
    quantidade_vendas,
    (receita - LAG(receita, 1, 0) OVER (ORDER BY mes_ano)) / LAG(receita, 1, 0) OVER (ORDER BY mes_ano) * 100 AS crescimento_receita_mensal_percentual,
    (receita - receita_ano_anterior) / receita_ano_anterior * 100 AS crescimento_receita_anual_percentual,
    (quantidade_vendas - LAG(quantidade_vendas, 1, 0) OVER (ORDER BY mes_ano)) / LAG(quantidade_vendas, 1, 0) OVER (ORDER BY mes_ano) * 100 AS crescimento_quantidade_mensal_percentual,
    (quantidade_vendas - quantidade_ano_anterior) / quantidade_ano_anterior * 100 AS crescimento_quantidade_anual_percentual
FROM comparacao_anual
ORDER BY mes_ano DESC
"""))


mes_ano,receita,quantidade_vendas,crescimento_receita_mensal_percentual,crescimento_receita_anual_percentual,crescimento_quantidade_mensal_percentual,crescimento_quantidade_anual_percentual
2024-07-01T00:00:00Z,21589.58,2560,-40.896400,-36.477500,-31.787903010924595,-16.175507531106746
2024-06-01T00:00:00Z,36528.39,3753,-57.947600,null,-51.617893515534355,null
2024-05-01T00:00:00Z,86863.91,7757,45.124600,null,45.04487658937921,null
2024-04-01T00:00:00Z,59854.70,5348,4.508500,null,7.324904675898053,null
2024-03-01T00:00:00Z,57272.58,4983,-2.787000,null,-17.280876494023907,null
2024-02-01T00:00:00Z,58914.52,6024,6.520600,null,16.069364161849713,null
2024-01-01T00:00:00Z,55308.10,5190,-3.493400,null,-15.472312703583063,null
2023-12-01T00:00:00Z,57310.15,6140,-14.196800,null,-2.24486546728228,null
2023-11-01T00:00:00Z,66792.56,6281,47.575500,null,76.6310461192351,null
2023-10-01T00:00:00Z,45259.91,3556,-44.326300,null,-48.04208065458796,null
